## LLM

In [57]:
import sys
sys.path.insert(0, '../src/')

import matplotlib.pyplot as plt
from tokenizer import TokenizerBPE
from data_handling import normalize_to_ascii, clean_text
from utils import saver, loader
import numpy as np
import re

import os
import pickle as pkl
from tqdm.notebook import tqdm
import json
import random

# disable gpu for testing purposes
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


## Urban Dictionary

In [5]:
data = []
with open('../corpus/words.json', 'r') as f:
    for line in tqdm(f):
        data.append(json.loads(line))


0it [00:00, ?it/s]

In [66]:
example_list = []
definition_list = []

tu_list = []
td_list = []

for entry in tqdm(data):
    thumbs_up = entry['thumbs_up']
    tu_list.append(thumbs_up)
    thumbs_down = entry['thumbs_down']
    td_list.append(thumbs_down)
    
    word = entry['word']
    definition = entry['definition']
    example = entry['example']

    if thumbs_up > 4*thumbs_down and thumbs_up > 100:
        word = clean_text(normalize_to_ascii(word))
        example = clean_text(normalize_to_ascii(example))
        definition = clean_text(normalize_to_ascii(definition))

        example_list.append(example)
        definition_concat = word + ': ' + definition + '. example: ' + example
        definition_list.append(definition_concat.replace('..', '.'))

  0%|          | 0/2606521 [00:00<?, ?it/s]

In [67]:
print(f'Number of examples: {len(example_list)}')
print(len("".join(definition_list)))

Number of examples: 45284
19857084


In [68]:
saver("../corpus/urbanDict_raw", [definition_list, example_list])

In [8]:
def fused_sqa(story_list, question_grouped_list, answer_grouped_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    rcw = re.compile(r"\s+")

    corpus_list = []
    for story, question_list, answer_list in tqdm(list(zip(story_list, question_grouped_list, answer_grouped_list))):
        sqa = [sos, normalize_to_ascii(story).lower()]
        for question, answer in zip(question_list, answer_list):
            sqa.append(q)
            sqa.append(normalize_to_ascii(question).lower())
            sqa.append(a)
            sqa.append(normalize_to_ascii(answer).lower())
        sqa.append(eos)
        corpus_list.append("".join(sqa))
        
    return corpus_list

In [11]:
corpus_list = fused_sqa(context_list, question_grouped, answer_grouped)

  0%|          | 0/7199 [00:00<?, ?it/s]

## 